Chia tỉ lệ 70-20-10 cho T-Students-Fitdnu



test xem có trùng ảnh + lables

In [3]:
from pathlib import Path

# Đường dẫn thư mục
base_path = Path("/hdd2/minhnv/CodingYOLOv12/Dataset/T-Student_FIT-DNU-update-1")
splits = ['train', 'valid', 'test']

# Hàm kiểm tra từng split
def check_split(split):
    images_dir = base_path / split / "images"
    labels_dir = base_path / split / "labels"

    image_files = list(images_dir.glob("*.[jp][pn]g"))  # jpg hoặc png
    label_files = list(labels_dir.glob("*.txt"))

    image_names = set([f.stem for f in image_files])
    label_names = set([f.stem for f in label_files])

    missing_labels = image_names - label_names
    missing_images = label_names - image_names

    if missing_labels:
        print(f"❌ [{split.upper()}] Có ảnh bị thiếu file nhãn: {missing_labels}")
    if missing_images:
        print(f"❌ [{split.upper()}] Có nhãn bị thiếu file ảnh: {missing_images}")
    if not missing_labels and not missing_images:
        print(f"✅ [{split.upper()}] Ảnh và nhãn khớp hoàn toàn. Tổng số: {len(image_files)} ảnh.")

    return len(image_files)

# Tổng số ảnh sau chia
total_images = 0
for s in splits:
    total_images += check_split(s)

# Kiểm tra thư mục train gốc đã trống chưa
old_images = list((base_path / "train/images").glob("*"))
old_labels = list((base_path / "train/labels").glob("*"))

if not old_images and not old_labels:
    print("✅ Thư mục train/images và train/labels cũ đã được dọn sạch.")
else:
    print("⚠️ Còn file chưa di chuyển hết trong train/images hoặc train/labels.")

print(f"📊 Tổng ảnh sau phân chia: {total_images}")


✅ [TRAIN] Ảnh và nhãn khớp hoàn toàn. Tổng số: 616 ảnh.
✅ [VALID] Ảnh và nhãn khớp hoàn toàn. Tổng số: 176 ảnh.
✅ [TEST] Ảnh và nhãn khớp hoàn toàn. Tổng số: 88 ảnh.
⚠️ Còn file chưa di chuyển hết trong train/images hoặc train/labels.
📊 Tổng ảnh sau phân chia: 880


Chia dữ liệu 70-20-10

In [2]:
import os
import random
import shutil
from pathlib import Path

# Đặt seed để kết quả random reproducible
random.seed(42)

# Đường dẫn thư mục ảnh gốc (train hiện tại)
source_images_dir = Path("/hdd2/minhnv/CodingYOLOv12/Dataset/T-Student_FIT-DNU-6/train/images")
source_labels_dir = Path("/hdd2/minhnv/CodingYOLOv12/Dataset/T-Student_FIT-DNU-6/train/labels")

# ✅ Thư mục đích mới
base_path = Path("/hdd2/minhnv/CodingYOLOv12/Dataset/T-Student_FIT-DNU-update-6-split")
dest_dirs = {
    "train": {
        "images": base_path / "train/images",
        "labels": base_path / "train/labels"
    },
    "val": {
        "images": base_path / "valid/images",
        "labels": base_path / "valid/labels"
    },
    "test": {
        "images": base_path / "test/images",
        "labels": base_path / "test/labels"
    }
}

# Tạo các thư mục nếu chưa có
for split in dest_dirs.values():
    split["images"].mkdir(parents=True, exist_ok=True)
    split["labels"].mkdir(parents=True, exist_ok=True)

# Lấy toàn bộ file ảnh (.jpg hoặc .png)
image_files = list(source_images_dir.glob("*.jpg")) + list(source_images_dir.glob("*.png"))
random.shuffle(image_files)

# Tính số lượng ảnh cho mỗi tập
total = len(image_files)
train_split = int(0.7 * total)
val_split = int(0.2 * total)

# Hàm sao chép ảnh + nhãn tương ứng sang thư mục mới
def copy_files(files, split_name):
    for img_path in files:
        label_path = source_labels_dir / (img_path.stem + ".txt")

        # Copy ảnh
        shutil.copy(str(img_path), dest_dirs[split_name]["images"] / img_path.name)

        # Copy nhãn nếu có
        if label_path.exists():
            shutil.copy(str(label_path), dest_dirs[split_name]["labels"] / label_path.name)

# Chia và sao chép dữ liệu
copy_files(image_files[:train_split], "train")
copy_files(image_files[train_split:train_split + val_split], "val")
copy_files(image_files[train_split + val_split:], "test")

print(f"✅ Đã phân bổ: {train_split} ảnh train, {val_split} ảnh val, {total - train_split - val_split} ảnh test.")
print(f"📂 Output lưu tại: {base_path}")


✅ Đã phân bổ: 4305 ảnh train, 1230 ảnh val, 615 ảnh test.
📂 Output lưu tại: /hdd2/minhnv/CodingYOLOv12/Dataset/T-Student_FIT-DNU-update-6-split
